<a href="https://colab.research.google.com/github/head1ton/llm_a_z_finetuning/blob/main/notebook/%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A1%9C%EB%B6%80%ED%84%B0_%EC%A3%BC%EC%9A%94_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OpenAI의 모델들의 이름들과 가격은 아래의 주소에서 확인하세요.

https://openai.com/pricing

# 1. 셋팅하기

코랩에서 OpenAI ChatGPT API를 실습하려면 openai 라이브러리를 설치해야 합니다. 다음 명령어를 입력합니다

In [ ]:
!pip install openai

마지막 부분과 같이 ‘Successfully installed openai’라는
문구를 확인했다면 import openai를 통해 라이브러리를 불러옵니다. 그
후 openai.api_key의 값으로 OpenAI API 키 발급하는 사이트(https://platform.openai.com/account/api-keys) 에서 얻은 키
값을 입력합니다.

In [40]:
from tqdm import tqdm
import openai

client = openai.OpenAI(api_key="여러분의 Key 값")

# 업무자동화

In [3]:
import pandas as pd
import ast

# 데이터를 리스트의 형태로 정리
data = {
    "수리일자": ["2023-09-07", "2021-10-08", "2022-02-23", "2022-04-06", "2021-02-20", "2021-02-26", "2021-04-13", "2021-07-30", "2022-07-12", "2022-07-13", "2022-07-18", "2022-07-18", "2022-08-11", "2022-09-13", "2022-10-04", "2022-10-26", "2022-11-11", "2022-11-14"],
    "고장내용": [
        "출하장 입구 에어 50A 유니온 에어리크 재조임",
        "SHIELD ROOM 화풍기 철거부위 마감,판넬벽체문타공",
        "F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정",
        "와이어 전진동작이상 점검 중 정상 작동됨",
        "클램프기동이상 오일보충 에어 제거",
        "피이더 조작판 매입 콘센트 고정",
        "QDC 파손 클램프 교체",
        "광세샤 고정블록 볼트 마모 600ton 것으로 대체",
        "휠 회전 작동불량 점검 세기정공 AS 요청",
        "클러치 ASS'Y 작동이상 분해출문(세기정공)",
        "클러치 assy 불량으로 전자접촉기교체",
        "클러치 밋션 기어오일보출 ATF-3",
        "휨 가지센서 단선 교체",
        "안전1행정 타발정지위치이상 리미트캠스위치교체및조정,상사점리미트스위치교체",
        "QDC 클램프 기동이상 에어빼기",
        "피이더 소재 진입이상 캠 스위치 조정",
        "카운터 완료시 정지시점 불량 카운타 각도 조정",
        "NC FEEDER 코일 이송거리 이상 재 셋팅"
    ],
    "고장부품": [None if x in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] else x for x in ["50A 유니온", "SHIELD ROOM 화풍기"] + [None]*16],
    "불량유형": [None if x in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] else x for x in ["에어리크", "철거부위 마감"] + [None]*16],
    "조치내용": [None if x in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] else x for x in ["재조임", "판넬벽체문타공"] + [None]*16]
}

# 데이터프레임 생성
df = pd.DataFrame(data)

In [4]:
# 두 개의 예시만 작성된 데이터프레임.
df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2023-09-07,출하장 입구 에어 50A 유니온 에어리크 재조임,50A 유니온,에어리크,재조임
1,2021-10-08,"SHIELD ROOM 화풍기 철거부위 마감,판넬벽체문타공",SHIELD ROOM 화풍기,철거부위 마감,판넬벽체문타공
2,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,None,None,None
3,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,None,None,None
4,2021-02-20,클램프기동이상 오일보충 에어 제거,None,None,None
5,2021-02-26,피이더 조작판 매입 콘센트 고정,None,None,None
6,2021-04-13,QDC 파손 클램프 교체,None,None,None
7,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,None,None,None
8,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,None,None,None
9,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),None,None,None


In [6]:
# 고장부품, 불량유형, 조치내용이 존재하지 않는 데이터만 남긴다.
sub_df = df[df['고장부품'].isnull()]

# 인덱스 번호를 초기화한다. (가장 왼쪽에 붙는 0부터 시작하는 번호)
sub_df.reset_index(drop=True, inplace=True)
sub_df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,None,None,None
1,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,None,None,None
2,2021-02-20,클램프기동이상 오일보충 에어 제거,None,None,None
3,2021-02-26,피이더 조작판 매입 콘센트 고정,None,None,None
4,2021-04-13,QDC 파손 클램프 교체,None,None,None
5,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,None,None,None
6,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,None,None,None
7,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),None,None,None
8,2022-07-18,클러치 assy 불량으로 전자접촉기교체,None,None,None
9,2022-07-18,클러치 밋션 기어오일보출 ATF-3,None,None,None


In [7]:
# 16개의 고장 내용을 파이썬 리스트로 변환
inputs= sub_df['고장내용'].to_list()
print(inputs)

['F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정', '와이어 전진동작이상 점검 중 정상 작동됨', '클램프기동이상 오일보충 에어 제거', '피이더 조작판 매입 콘센트 고정', 'QDC 파손 클램프 교체', '광세샤 고정블록 볼트 마모 600ton 것으로 대체', '휠 회전 작동불량 점검 세기정공 AS 요청', "클러치 ASS'Y 작동이상 분해출문(세기정공)", '클러치 assy 불량으로 전자접촉기교체', '클러치 밋션 기어오일보출 ATF-3', '휨 가지센서 단선 교체', '안전1행정 타발정지위치이상 리미트캠스위치교체및조정,상사점리미트스위치교체', 'QDC 클램프 기동이상 에어빼기', '피이더 소재 진입이상 캠 스위치 조정', '카운터 완료시 정지시점 불량 카운타 각도 조정', 'NC FEEDER 코일 이송거리 이상 재 셋팅']


In [41]:
# 고장내용으로부터 고장부품, 불량유형, 조치내용을 반환하는 함수 작성.
# 예시를 작성하여 답변의 안정성을 높이고 추후 파싱이 편하도록 파이썬의 Dictionary 형태로 답변을 요구한다.
def return_label(text):
    prompt = '''주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
    이때 파이썬의 Dictionary 형태로 반환하시오.
    마크다운은 사용하지 마십시오.

    ex)
    고장내용: '출하장 입구 에어 50A 유니온 에어리크 재조임'
    출력: {"고장부품": "50A 유니온", "불량유형" : "에어리크", "조치내용" : "재조임"}

    고장내용: 'SHIELD ROOM 화풍기 철거부위 마감,판넬벽체문타공	'
    출력: {"고장부품": "SHIELD ROOM 화풍기", "불량유형" : "철거부위 마감", "조치내용" : "판넬벽체문타공"}'''

    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": prompt},
          {"role": "user", "content": "고장내용: " + text + "\n출력: "}
      ],
    temperature=0
    )
    return response.choices[0].message.content

In [42]:
# 임의의 입력
test = sub_df['고장내용'].loc[0]
print(test)

F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정


In [44]:
return_label(test)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# 임의의 입력
test = sub_df['고장내용'].loc[1]
print(test)

와이어 전진동작이상 점검 중 정상 작동됨


In [ ]:
return_label(test)

'{"고장부품": "와이어", "불량유형": "전진동작이상", "조치내용": "점검 중 정상 작동됨"}'

In [ ]:
# 16개의 고장 내용에 대해서 반복 호출.
result = []
for input in tqdm(sub_df['고장내용'].to_list()):
  result.append(return_label(input))

100%|██████████| 16/16 [00:17<00:00,  1.08s/it]


In [ ]:
result

['{"고장부품": "F/P", "불량유형": "기능상실", "조치내용": "PLC 확인 및 클램프 감지센샤 조정"}',
 '{"고장부품": "와이어", "불량유형": "전진동작이상", "조치내용": "점검 중 정상 작동됨"}',
 '{"고장부품": "클램프", "불량유형": "기동이상", "조치내용": "오일보충 에어 제거"}',
 '{"고장부품": "피이더 조작판", "불량유형": "매입 콘센트", "조치내용": "고정"}',
 '{"고장부품": "QDC", "불량유형": "파손", "조치내용": "클램프 교체"}',
 '{"고장부품": "고정블록 볼트", "불량유형": "마모", "조치내용": "600ton 것으로 대체"}',
 '{"고장부품": "휠", "불량유형": "회전 작동불량", "조치내용": "점검 세기정공 AS 요청"}',
 '{"고장부품": "클러치 ASS\'Y", "불량유형": "작동이상", "조치내용": "분해출문(세기정공)"}',
 '{"고장부품": "클러치 assy", "불량유형": "불량", "조치내용": "전자접촉기교체"}',
 '{"고장부품": "클러치 밋션", "불량유형": "기어오일보출", "조치내용": "ATF-3"}',
 '{"고장부품": "휨 가지센서", "불량유형": "단선", "조치내용": "교체"}',
 '{"고장부품": "안전1행정", "불량유형": "타발정지위치이상", "조치내용": "리미트캠스위치교체및조정, 상사점리미트스위치교체"}',
 '{"고장부품": "QDC 클램프", "불량유형": "기동이상", "조치내용": "에어빼기"}',
 '{"고장부품": "피이더 소재", "불량유형": "진입이상", "조치내용": "캠 스위치 조정"}',
 '{"고장부품": "카운터", "불량유형": "정지시점 불량", "조치내용": "각도 조정"}',
 '{"고장부품": "NC FEEDER 코일", "불량유형": "이송거리 이상", "조치내용": "재 셋팅"}']

In [ ]:
# GPT-4가 작성한 데이터 중 첫번째 데이터를 출력
result[0]

'{"고장부품": "F/P", "불량유형": "기능상실", "조치내용": "PLC 확인 및 클램프 감지센샤 조정"}'

In [ ]:
# GPT-4가 작성한 데이터는 생긴 것은 파이썬 Dictionary처럼 생겼지만 실제로는 문자열 타입이다.
print(type(result[0]))

<class 'str'>


In [ ]:
# 16개의 데이터를 문자열 타입을 파이썬 Dictionary 형태로 전부 변환
result = [ast.literal_eval(elem) for elem in result]
print(result)

[{'고장부품': 'F/P', '불량유형': '기능상실', '조치내용': 'PLC 확인 및 클램프 감지센샤 조정'}, {'고장부품': '와이어', '불량유형': '전진동작이상', '조치내용': '점검 중 정상 작동됨'}, {'고장부품': '클램프', '불량유형': '기동이상', '조치내용': '오일보충 에어 제거'}, {'고장부품': '피이더 조작판', '불량유형': '매입 콘센트', '조치내용': '고정'}, {'고장부품': 'QDC', '불량유형': '파손', '조치내용': '클램프 교체'}, {'고장부품': '고정블록 볼트', '불량유형': '마모', '조치내용': '600ton 것으로 대체'}, {'고장부품': '휠', '불량유형': '회전 작동불량', '조치내용': '점검 세기정공 AS 요청'}, {'고장부품': "클러치 ASS'Y", '불량유형': '작동이상', '조치내용': '분해출문(세기정공)'}, {'고장부품': '클러치 assy', '불량유형': '불량', '조치내용': '전자접촉기교체'}, {'고장부품': '클러치 밋션', '불량유형': '기어오일보출', '조치내용': 'ATF-3'}, {'고장부품': '휨 가지센서', '불량유형': '단선', '조치내용': '교체'}, {'고장부품': '안전1행정', '불량유형': '타발정지위치이상', '조치내용': '리미트캠스위치교체및조정, 상사점리미트스위치교체'}, {'고장부품': 'QDC 클램프', '불량유형': '기동이상', '조치내용': '에어빼기'}, {'고장부품': '피이더 소재', '불량유형': '진입이상', '조치내용': '캠 스위치 조정'}, {'고장부품': '카운터', '불량유형': '정지시점 불량', '조치내용': '각도 조정'}, {'고장부품': 'NC FEEDER 코일', '불량유형': '이송거리 이상', '조치내용': '재 셋팅'}]


In [ ]:
# 이제 Dictionary 형태로 바뀌었다.
print(type(result[0]))

<class 'dict'>


In [ ]:
# Dictionary의 리스트 형태를 파이썬 데이터프레임으로 변환
new_df = pd.DataFrame(result)

In [ ]:
new_df

,고장부품,불량유형,조치내용
0,F/P,기능상실,PLC 확인 및 클램프 감지센샤 조정
1,와이어,전진동작이상,점검 중 정상 작동됨
2,클램프,기동이상,오일보충 에어 제거
3,피이더 조작판,매입 콘센트,고정
4,QDC,파손,클램프 교체
5,고정블록 볼트,마모,600ton 것으로 대체
6,휠,회전 작동불량,점검 세기정공 AS 요청
7,클러치 ASS'Y,작동이상,분해출문(세기정공)
8,클러치 assy,불량,전자접촉기교체
9,클러치 밋션,기어오일보출,ATF-3


In [ ]:
sub_df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,None,None,None
1,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,None,None,None
2,2021-02-20,클램프기동이상 오일보충 에어 제거,None,None,None
3,2021-02-26,피이더 조작판 매입 콘센트 고정,None,None,None
4,2021-04-13,QDC 파손 클램프 교체,None,None,None
5,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,None,None,None
6,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,None,None,None
7,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),None,None,None
8,2022-07-18,클러치 assy 불량으로 전자접촉기교체,None,None,None
9,2022-07-18,클러치 밋션 기어오일보출 ATF-3,None,None,None


In [ ]:
new_df

,고장부품,불량유형,조치내용
0,F/P,기능상실,PLC 확인 및 클램프 감지센샤 조정
1,와이어,전진동작이상,점검 중 정상 작동됨
2,클램프,기동이상,오일보충 에어 제거
3,피이더 조작판,매입 콘센트,고정
4,QDC,파손,클램프 교체
5,고정블록 볼트,마모,600ton 것으로 대체
6,휠,회전 작동불량,점검 세기정공 AS 요청
7,클러치 ASS'Y,작동이상,분해출문(세기정공)
8,클러치 assy,불량,전자접촉기교체
9,클러치 밋션,기어오일보출,ATF-3


In [ ]:
sub_df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,None,None,None
1,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,None,None,None
2,2021-02-20,클램프기동이상 오일보충 에어 제거,None,None,None
3,2021-02-26,피이더 조작판 매입 콘센트 고정,None,None,None
4,2021-04-13,QDC 파손 클램프 교체,None,None,None
5,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,None,None,None
6,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,None,None,None
7,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),None,None,None
8,2022-07-18,클러치 assy 불량으로 전자접촉기교체,None,None,None
9,2022-07-18,클러치 밋션 기어오일보출 ATF-3,None,None,None


In [ ]:
sub_df['고장부품'] = new_df['고장부품']
sub_df['불량유형'] = new_df['불량유형']
sub_df['조치내용'] = new_df['조치내용']

<ipython-input-48-2e3c98d9f99a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['고장부품'] = new_df['고장부품']
<ipython-input-48-2e3c98d9f99a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['불량유형'] = new_df['불량유형']
<ipython-input-48-2e3c98d9f99a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [ ]:
sub_df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,F/P,기능상실,PLC 확인 및 클램프 감지센샤 조정
1,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,와이어,전진동작이상,점검 중 정상 작동됨
2,2021-02-20,클램프기동이상 오일보충 에어 제거,클램프,기동이상,오일보충 에어 제거
3,2021-02-26,피이더 조작판 매입 콘센트 고정,피이더 조작판,매입 콘센트,고정
4,2021-04-13,QDC 파손 클램프 교체,QDC,파손,클램프 교체
5,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,고정블록 볼트,마모,600ton 것으로 대체
6,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,휠,회전 작동불량,점검 세기정공 AS 요청
7,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),클러치 ASS'Y,작동이상,분해출문(세기정공)
8,2022-07-18,클러치 assy 불량으로 전자접촉기교체,클러치 assy,불량,전자접촉기교체
9,2022-07-18,클러치 밋션 기어오일보출 ATF-3,클러치 밋션,기어오일보출,ATF-3


# ESG 판단 모델

모델에게 답을 작성할 때 `근거`를 작성하고나서 결론을 내리라고 하는 방식을 `생각의 사슬(Chain of Thougt) 프롬프팅`이라고 한다.  

이 방식을 사용하면 실제로 모델의 답변의 정확도가 높아질 뿐만 아니라 학습 모델의 판단 근거를 사용자가 확인할 수 있다는 장점이 있다.

In [ ]:
system_prompt = '''당신은 ESG에 대해서 점수를 계산하는 모델입니다. 평가 기준은 다음과 같습니다.

## 지시사항
주어진 평가 기준을 바탕으로 뉴스 기사로부터 환경, 사회, 지배 구조에 대해서 점수를 측정하십시오.

## 평가 기준
환경
1점: 환경 오염 유발, 법적 제재, 규제 위반 등의 사건이 발생한 경우. 예: 심각한 오염 사고, 규제 위반으로 인한 벌금 부과
2점: 환경 보호에 대한 소극적인 태도 또는 일회성 활동에 그치는 경우. 예: 임시적인 환경 캠페인 참여, 적극적인 감축 목표 미설정
3점: 환경 보호를 위한 노력이 있으나 체계적인 전략이 부족한 경우. 예: 온실가스 감축 계획 발표, 일부 친환경 인증 획득
4점: 체계적인 환경 관리 계획이 있으며, 목표 달성에 꾸준히 투자하는 경우. 예: 지속 가능한 에너지 사용, 체계적인 폐기물 관리 프로그램 운영
5점: 적극적인 환경 정책과 뛰어난 실적 결과를 보이며 업계 선도적 활동을 하는 경우. 예: 탄소 배출 네트 제로(Net Zero) 목표 실현, 친환경 기술 선도

사회
1점: 인권, 근로자 안전 문제 발생, 지역 사회와의 갈등 등 부정적 이슈 발생. 예: 직원 착취 문제, 안전 관리 소홀로 인한 사고
2점: 직원 복지나 지역 사회 기여에 대한 제한적 활동 또는 단기적인 참여. 예: 일회성 기부, 사내 복지 프로그램 부족
3점: 직원 복지 및 사회적 기여에 관심을 가지며 일부 프로그램을 운영하는 경우. 예: 사내 복지 프로그램 운영, 특정 지역 사회 활동 참여
4점: 직원 복지, 다양성, 지역 사회 공헌에 적극적으로 나서며 책임감 있게 운영하는 경우. 예: 다양성 강화 프로그램, 정기적인 사회공헌 활동
5점: 사회적 가치 창출에 지속적으로 기여하며 업계 모범적인 활동을 보여주는 경우. 예: 폭넓은 사회적 책임 활동, 소외 계층 지원 프로그램

지배구조
1점: 지배구조의 투명성 부족, 부패 사건 또는 규제 위반이 발생한 경우. 예: 부패 스캔들, 재무 조작
2점: 지배구조 개선 노력이 부족하며, 외부 감사나 투명성 문제에 있는 경우. 예: 의사 결정 과정 불투명, 주주와의 소통 부족
3점: 기본적인 지배구조 원칙을 따르나, 일부 개선의 여지가 있는 경우. 예: 주주와의 소통 강화 중, 투명성 개선 노력
4점: 투명하고 윤리적인 지배구조를 실천하고 있으며, 책임 있는 경영을 하는 경우. 예: 정기적인 외부 감사, 투명한 의사결정 구조
5점: 업계에서 모범이 되는 윤리적 지배구조를 유지하고 지속적 개선을 보이는 경우. 예: 독립적 이사회 구성, 주주 및 이해관계자와의 적극적 소통

## 답변 형식
답변 형식은 반드시 아래의 형식을 따르십시오. 근거:, E:, S:, G: 외에는 그 어떠한 문장도 덧붙이지 마십시오.

근거: 해당 뉴스 기사에 대해서 ESG 점수를 작성한 근거를 작성합니다.
E: 1~5 사이의 정수
S: 1~5 사이의 정수
G: 1~5 사이의 정수

답변 형식 예시는 다음과 같습니다.
근거: 이 뉴스 기사는 방화6구역 조합과 HDC현대산업개발 간의 공사비 갈등과 시공사 계약 해지 과정을 다루고 있습니다. 주 내용은 공사비 인상 문제로 인한 갈등이 심화되어 결국 시공사 교체를 논의하는 상황에 놓여 있다는 것입니다. 이로 인해 사회적 이슈로는 공사 중단 사태가 장기화되고 법적 분쟁까지 이어졌습니다. 환경에 대한 명시적 내용은 없으며, 지배구조와 관련되어 시공사와 조합 간의 협상 실패와 조합원 반발이 드러나고 있습니다.
E: 2
S: 1
G: 2

## 뉴스 기사'''

In [ ]:
user_prompt = "LG家 장녀 ‘주식 부정거래’ 의혹, 금융당국 덜미 잡았다 금감원 [사진 = 연합뉴스]  구연경 LG복지재단 대표가 미공개정보를 이용해 코스닥 상장사 주식을 매수했다는 의혹과 관련해 금융당국이 관련 정황을 포착하고 제재에 착수했다. 구 대표는 고(故) 구본무 LG그룹 선대회장의 장녀다.  27일 금융당국에 따르면 금융감독원은 구 대표가 지난해 코스닥 상장사인 바이오기업 A사 주식을 해당 회사의 투자유치 정보가 알려지기 전에 취득한 것으로 보고 자본시장법 위반 혐의로 해당 안건을 금융위원회에 넘겼다.  금융위는 해당 안건을 내달 2일 열리는 증권선물위원회에 올려 처리할 예정이다. 증선위 결정에 따라 구 대표 관련 사건은 검찰 등 수사기관에 고발 또는 통보될 것으로 보인다.  금감원 고위관계자는 “본인은 혐의를 부인하고 있지만, 거래시점상 미공개정보를 이용했다는 정황증거를 포착했다”며 “의혹이 맞다면 형사처벌 대상이 되는 만큼 검찰 등 수사기관의 수사가 필요한 상황”이라고 설명했다.  앞서 지난해 4월 A사는 BRV(블루런벤처스)캐피탈매니지먼트로부터 제3자 배정 유상증자를 통해 500억원을 조달했다. BRV캐피탈매니지먼트는 실리콘밸리 기반 기관투자사 블루런벤처스의 글로벌 성장 투자 플랫폼이다. 구 대표의 남편인 윤관씨는 BRV캐피탈매니지먼트의 최고투자책임자(CIO)를 맡고 있다.  금융당국은 A사의 자금조달 계획이 알려지기 전 구 대표를 이를 미리 알고 그 전에 본인과 관련인 계좌 등을 이용해 해당 회사 주식 3만주를 개인적으로 취득한 것으로 보고 있다.  만약 구 대표가 주식을 사들인 게 투자 발표 전일 경우, 일반 주주는 알 수 없었던 투자건이라는 미공개 정보를 활용한 자본시장법 위반 행위가 된다.  지난해 3월말 주당 1만6000원 수준이던 A사 주가는 BRV캐피탈머니지먼트의 투자사실이 알려진 당일에만 16% 이상 급등했고, 지난해 한때는 5만원까지 치솟았다.  자본시장법 제174조는 상장법인의 업무 등과 관련한 미공개 중요정보를 특정 증권 등의 매매·거래에 이용하는 행위를 금지하고 있다. 이를 위반하면 1년 이상의 유기징역 또는 그 행위로 얻은 이익 또는 회피한 손실액의 3배 이상 5배 이하에 상당하는 벌금이 매겨진다.  앞서 금감원은 A사의 기타비상무이사(등기임원)를 맡고 있는 B씨를 소환조사한 바 있다. 이후 추가 조사를 통해 구씨의 혐의를 확인한 것으로 보인다.  한편 구 대표는 A사 주식 취득건이 논란이 되자 지난 5월 해당 주식을 LG복지재단에 기부하려고 시도했다. 다만 당시 재단 이사회는 미공개 정보를 활용한 주식 매수가 의심된다는 이유로 관련 안건을 처리하지 않아 실제 기부는 이뤄지지 않았다."

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt + '\n근거:'}
    ]
)

In [ ]:
print(response.choices[0].message.content)

근거: 이 뉴스 기사는 LG복지재단 대표의 주식 부정거래 의혹과 관련된 금융당국의 조사에 대해 다루고 있습니다. 환경 문제와 관련된 내용은 없으며, 사회적 측면에서는 부정 거래 의혹이 사회에 미치는 영향을 고려할 수 있습니다. 지배구조와 관련해서는 금융규제 및 지배구조의 투명성 문제에 대한 내용이 포함되어 있습니다.
E: 2
S: 2
G: 1


그렇다면 근거를 작성해달라는 말을 취소하고 요청하면 어떨까?

In [ ]:
system_prompt = '''당신은 ESG에 대해서 점수를 계산하는 모델입니다. 평가 기준은 다음과 같습니다.

## 지시사항
주어진 평가 기준을 바탕으로 뉴스 기사로부터 환경, 사회, 지배 구조에 대해서 점수를 측정하십시오.

## 평가 기준
환경
1점: 환경 오염 유발, 법적 제재, 규제 위반 등의 사건이 발생한 경우. 예: 심각한 오염 사고, 규제 위반으로 인한 벌금 부과
2점: 환경 보호에 대한 소극적인 태도 또는 일회성 활동에 그치는 경우. 예: 임시적인 환경 캠페인 참여, 적극적인 감축 목표 미설정
3점: 환경 보호를 위한 노력이 있으나 체계적인 전략이 부족한 경우. 예: 온실가스 감축 계획 발표, 일부 친환경 인증 획득
4점: 체계적인 환경 관리 계획이 있으며, 목표 달성에 꾸준히 투자하는 경우. 예: 지속 가능한 에너지 사용, 체계적인 폐기물 관리 프로그램 운영
5점: 적극적인 환경 정책과 뛰어난 실적 결과를 보이며 업계 선도적 활동을 하는 경우. 예: 탄소 배출 네트 제로(Net Zero) 목표 실현, 친환경 기술 선도

사회
1점: 인권, 근로자 안전 문제 발생, 지역 사회와의 갈등 등 부정적 이슈 발생. 예: 직원 착취 문제, 안전 관리 소홀로 인한 사고
2점: 직원 복지나 지역 사회 기여에 대한 제한적 활동 또는 단기적인 참여. 예: 일회성 기부, 사내 복지 프로그램 부족
3점: 직원 복지 및 사회적 기여에 관심을 가지며 일부 프로그램을 운영하는 경우. 예: 사내 복지 프로그램 운영, 특정 지역 사회 활동 참여
4점: 직원 복지, 다양성, 지역 사회 공헌에 적극적으로 나서며 책임감 있게 운영하는 경우. 예: 다양성 강화 프로그램, 정기적인 사회공헌 활동
5점: 사회적 가치 창출에 지속적으로 기여하며 업계 모범적인 활동을 보여주는 경우. 예: 폭넓은 사회적 책임 활동, 소외 계층 지원 프로그램

지배구조
1점: 지배구조의 투명성 부족, 부패 사건 또는 규제 위반이 발생한 경우. 예: 부패 스캔들, 재무 조작
2점: 지배구조 개선 노력이 부족하며, 외부 감사나 투명성 문제에 있는 경우. 예: 의사 결정 과정 불투명, 주주와의 소통 부족
3점: 기본적인 지배구조 원칙을 따르나, 일부 개선의 여지가 있는 경우. 예: 주주와의 소통 강화 중, 투명성 개선 노력
4점: 투명하고 윤리적인 지배구조를 실천하고 있으며, 책임 있는 경영을 하는 경우. 예: 정기적인 외부 감사, 투명한 의사결정 구조
5점: 업계에서 모범이 되는 윤리적 지배구조를 유지하고 지속적 개선을 보이는 경우. 예: 독립적 이사회 구성, 주주 및 이해관계자와의 적극적 소통

## 답변 형식
답변 형식은 반드시 아래의 형식을 따르십시오. E:, S:, G: 외에는 그 어떠한 문장도 덧붙이지 마십시오.
E: 1~5 사이의 정수
S: 1~5 사이의 정수
G: 1~5 사이의 정수

답변 형식 예시는 다음과 같습니다.
E: 2
S: 1
G: 2

## 뉴스 기사'''

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)

In [ ]:
print(response.choices[0].message.content)

E: 3
S: 3
G: 1


LLM 파인 튜닝에 관심이 많다면 ChatGPT API를 이용하여 모델을 학습하기 위한 데이터를 만드는 연습이 많이 필요하다.